In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import os
import gmaps
import gmplot
from geolocation.main import GoogleMaps
from geolocation.distance_matrix.client import DistanceMatrixApiClient
from yelpapi import YelpAPI
from pprint import pprint
from config import apikey
from config import gkey
import csv
from yelp.client import Client

In [2]:
# Get yo' Key
yelp_api = YelpAPI(apikey)
#print(yelp_api)

In [3]:
# Print each city's input into different lists. 
# We also need information on the restaurant's name, rating, location, lat/long and category
# To do this we're going to set up a list of cities, and a dictionary to hold it all in.
yelp_api = YelpAPI(apikey)
city_names = ["Oceanside", "Encinitas", "Carlsbad"]
data = {
    city: {
        'names': [],
        'ratings': [],
        'categories': [],
        'location':[],
        'lat':[],
        'lng':[]
    }
    for city in city_names
}
# Then we're gonna print just to see that our lists and dictionaries are set up correctly
print(json.dumps(data, indent=4))

#Now we just run a loop through the city names to append the data into the city dictionaries where they belong
for city in city_names:
    search_results = yelp_api.search_query(location=city, 
                                           term="restaurant", 
                                           categories='japanese,asian,fusion,sushi,hibachi',
                                           limit = 50) 
                                           #offset=100)
    for b in search_results['businesses']:
        try:
            data[city]['names'].append(b["name"])
            data[city]['ratings'].append(b["rating"])
            data[city]['categories'].append(b["categories"][0]['title']+"/"+b["categories"][0]["alias"])
            data[city]['location'].append(b['location']['city'])
            data[city]['lat'].append(b['coordinates']['latitude'])
            data[city]['lng'].append(b['coordinates']['longitude'])
        except:
            data[city]['names'].append(np.nan)
            data[city]['ratings'].append(np.nan)
            data[city]['categories'].append(np.nan)
            data[city]['location'].append(np.nan)
            data[city]['lat'].append(np.nan)
            data[city]['lng'].append(np.nan)

# You can run a test here for length checks.

# print(len(search_results['businesses']))
# print(len(name))
# print(len(rating))
# print(len(category))
# print(len(city_list))

# Then we pretty print the dictionaries to make sure they're all populated
pprint(data, indent=4)

{
    "Oceanside": {
        "names": [],
        "ratings": [],
        "categories": [],
        "location": [],
        "lat": [],
        "lng": []
    },
    "Encinitas": {
        "names": [],
        "ratings": [],
        "categories": [],
        "location": [],
        "lat": [],
        "lng": []
    },
    "Carlsbad": {
        "names": [],
        "ratings": [],
        "categories": [],
        "location": [],
        "lat": [],
        "lng": []
    }
}
{   'Carlsbad': {   'categories': [   'Sushi Bars/sushi',
                                      'Ramen/ramen',
                                      'Asian Fusion/asianfusion',
                                      'Japanese/japanese',
                                      'Sushi Bars/sushi',
                                      'Japanese/japanese',
                                      'Sushi Bars/sushi',
                                      'Japanese/japanese',
                                      'Japanese/japanese'

In [18]:
# Now that we have everything in seperate dictionaries, let's put it together into 3 seperate dataframes
Oceanside_df = pd.DataFrame.from_dict(data["Oceanside"])
Carlsbad_df = pd.DataFrame.from_dict(data["Carlsbad"])
Encinitas_df = pd.DataFrame.from_dict(data["Encinitas"])

# We'll print the numbers to make sure they're at 50 each
print(len(Oceanside_df))
print(len(Carlsbad_df))
print(len(Encinitas_df))

# Then we're dropping any null values
Oceanside_df = Oceanside_df.dropna(how='any')
Carlsbad_df = Carlsbad_df.dropna(how='any')
Encinitas_df = Encinitas_df.dropna(how='any')

# And printing again to make sure those are dropped if there are any
print(len(Oceanside_df))
print(len(Carlsbad_df))
print(len(Encinitas_df))



50
50
50
50
50
50


In [19]:
# Since some of our data isn't from the cities we're interested in, let's fix that.

# First we'll set the index to "location"
Oceanside_df = Oceanside_df.set_index("location")
Carlsbad_df = Carlsbad_df.set_index("location")
Encinitas_df= Encinitas_df.set_index("location")

# Then we're going to run a loc on the cities we want
# This will make sure we only have the cities we are interested in
Oceanside_df=Oceanside_df.loc["Oceanside",:]
Carlsbad_df=Carlsbad_df.loc["Carlsbad",:]
Encinitas_df=Encinitas_df.loc["Encinitas",:]

# We'll print the lengths to see what's left, and the head to make sure the DFs are what we want

print(len(Oceanside_df))
print(len(Carlsbad_df))
print(len(Encinitas_df))

Oceanside_df.head(20)

20
16
13


,names,ratings,categories,lat,lng
location,,,,,
Oceanside,Maru Restaurant,4.0,Japanese/japanese,33.225033,-117.333364
Oceanside,Wrench & Rodent Seabasstropub,4.5,American (New)/newamerican,33.174645,-117.361908
Oceanside,Wulan Ramen Tap House,4.0,Ramen/ramen,33.237943,-117.297560
Oceanside,Ramen MENMA - Basso Drillman Oceanside,4.0,Ramen/ramen,33.195980,-117.378940
Oceanside,Harney Sushi,4.0,Sushi Bars/sushi,33.194667,-117.380578
Oceanside,Teri Cafe,4.0,Ramen/ramen,33.185647,-117.327160
Oceanside,Yukiya Sushi,4.0,Sushi Bars/sushi,33.181649,-117.335375
Oceanside,Sushi4Reel,4.5,Sushi Bars/sushi,33.216325,-117.271782
Oceanside,Teri Cafe,4.0,Sushi Bars/sushi,33.181676,-117.293104


In [38]:
# Let's see who our top 5 competitors are in each city
# We'll start with Oceanside
SortedOceanside=Oceanside_df.sort_values("ratings", ascending=False)
SortedCarlsbad=Carlsbad_df.sort_values("ratings",ascending=False)
SortedEncinitas=Encinitas_df.sort_values("ratings",ascending=False)

Top5Oceanside = SortedOceanside.head(5).reset_index()
Top5Oceanside

,location,names,ratings,categories,lat,lng
0,Oceanside,Sunshine Poke & Sushi Rolls,4.5,Japanese/japanese,33.196600,-117.379944
1,Oceanside,Rising Sun Sushi Harbor,4.5,Sushi Bars/sushi,33.204395,-117.389962
2,Oceanside,Sushi4Reel,4.5,Sushi Bars/sushi,33.216325,-117.271782
3,Oceanside,Wrench & Rodent Seabasstropub,4.5,American (New)/newamerican,33.174645,-117.361908
4,Oceanside,Maru Restaurant,4.0,Japanese/japanese,33.225033,-117.333364


In [39]:
Top5Carlsbad = SortedCarlsbad.head(5).reset_index()
Top5Carlsbad

,location,names,ratings,categories,lat,lng
0,Carlsbad,Yoshino Japanese Deli,4.5,Japanese/japanese,33.160316,-117.349518
1,Carlsbad,Hooked on Sushi,4.5,Sushi Bars/sushi,33.180475,-117.325042
2,Carlsbad,Mikko Sushi,4.5,Sushi Bars/sushi,33.162488,-117.343037
3,Carlsbad,The Land & Water Co,4.0,Sushi Bars/sushi,33.158797,-117.350856
4,Carlsbad,Blue Ocean Robata & Sushi Bar,4.0,Asian Fusion/asianfusion,33.161418,-117.346827


In [41]:
Top5Encinitas = SortedEncinitas.head(5).reset_index()
Top5Encinitas

,location,names,ratings,categories,lat,lng
0,Encinitas,Beach Bowls San Elijo,5.0,Japanese/japanese,33.021111,-117.284619
1,Encinitas,Kai Ola Sushi,4.5,Sushi Bars/sushi,33.064902,-117.302545
2,Encinitas,Rooster Ramen,4.5,Ramen/ramen,33.064490,-117.302569
3,Encinitas,Kaito Sushi,4.5,Japanese/japanese,33.047214,-117.259754
4,Encinitas,Hooked On Sushi,4.5,Japanese/japanese,33.052633,-117.261493


In [6]:
# Now we do heatmaps. First we're loading up the figures we want
gmaps.configure(api_key=gkey)
Carlsbad_coordinates = (33.1580933,-117.339839)
Carlsbad_fig=gmaps.figure(center=Carlsbad_coordinates, zoom_level=14)
Carlsbad_fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
Oceanside_coordinates = (33.1959, -117.3795)
Oceanside_fig = gmaps.figure(center=Oceanside_coordinates, zoom_level=14)
Oceanside_fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
Encinitas_coordinates = (33.0370, -117.2920)
Encinitas_fig = gmaps.figure(center=Encinitas_coordinates, zoom_level=14)
Encinitas_fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Then we make the variables for our heat points and weight
Carlsbad_locations = Carlsbad_df[["lat","lng"]].astype(float)
Carlsbad_ratings = Carlsbad_df["ratings"].astype(float)
Oceanside_locations = Oceanside_df[["lat","lng"]].astype(float)
Oceanside_ratings = Oceanside_df["ratings"].astype(float)
Encinitas_locations = Encinitas_df[["lat","lng"]].astype(float)
Encinitas_ratings = Encinitas_df["ratings"].astype(float)

In [10]:
# Now we set up the heat layer for each map and add it to their maps
Carlsbad_heat = gmaps.heatmap_layer(Carlsbad_locations, weights=Carlsbad_ratings, dissipating=False, max_intensity=5, point_radius=.001)
Carlsbad_fig.add_layer(Carlsbad_heat)
Carlsbad_fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
Oceanside_heat = gmaps.heatmap_layer(Oceanside_locations, weights=Oceanside_ratings, dissipating=False, max_intensity=5, point_radius=.001)
Oceanside_fig.add_layer(Oceanside_heat)
Oceanside_fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
Encinitas_heat = gmaps.heatmap_layer(Encinitas_locations, weights=Encinitas_ratings, dissipating=False, max_intensity=5, point_radius=.001)
Encinitas_fig.add_layer(Encinitas_heat)
Encinitas_fig

Figure(layout=FigureLayout(height='420px'))